# Upload Predations to Parenthood Server
Amy Jin

July 6th, 2018

This part is to push ML model predictions to database. 

### Connect to & Push Data to Parenthood Server

In [62]:
# global import
import os
import sys
import numpy as np
import pandas as pd
from collections import Counter
import operator
import time

# Connect to Parenthood server
import mysql.connector
import sshtunnel
import pureyaml

# handle path
project_dir = !pwd  # dir of current script/notebook file
config_file = open(project_dir[0] + "/db.yaml")
config = pureyaml.load(config_file.read())
config_file2 = open(project_dir[0] + "/db2.yaml")
config2 = pureyaml.load(config_file2.read())

# argument dictionary for sshtunnel
ssh_config = {
    'ssh_address_or_host': ('parenthood.set.care', 22),
    'ssh_username':        config['ssh_username'],
    'ssh_password':        config['ssh_password'],
    'remote_bind_address': ('127.0.0.1', 3306),
    'local_bind_address':  ('0.0.0.0', 3333),
}

# argument dictionary for mysql.connector
mysql_config = {
    'user':     config['mysql_user'],
    'password': config['mysql_passwd'],
    'host':     config['mysql_host'],
    'database': 'patch',
    'port':     3333,
}

In [61]:
# Create dataframe
import pandas as pd
import numpy as np

np.random.seed(0)
number_of_samples = 10
test_df_2 = pd.DataFrame({
    'feature1': np.random.random(number_of_samples),
    'feature2': np.random.random(number_of_samples),
    'class':    np.random.binomial(2, 0.1, size=number_of_samples),
    },columns=['feature1','feature2','class'])

print(test_df_2)

   feature1  feature2  class
0  0.548814  0.791725      1
1  0.715189  0.528895      0
2  0.602763  0.568045      0
3  0.544883  0.925597      0
4  0.423655  0.071036      0
5  0.645894  0.087129      0
6  0.437587  0.020218      0
7  0.891773  0.832620      1
8  0.963663  0.778157      0
9  0.383442  0.870012      0


In [64]:
with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
    print('SSH tunneling successful on port: {}'.format(tunnel.local_bind_port))
    connection = mysql.connector.connect(**mysql_config)
    cur = connection.cursor()
    print('MySQL server connected successfully!')
    
    # Import dataframe into MySQL
    import sqlalchemy
    database_username = config2['database_username']
    database_password = config2['database_password']
    database_ip       = config2['database_ip']
    database_name     = 'ml_provider_type'
    database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                   format(database_username, database_password, 
                                                          database_ip, database_name))
    test_df_2.to_sql(con=database_connection, name='test_table', if_exists='replace')
    print('Data has been uploaded to MySQL server successfully!')

SSH tunneling successful on port: 3333
MySQL server connected successfully!
Data has been uploaded to MySQL server successfully!


In [55]:
def UploadPredictionsToParenthood(db_name, db_table_name, table_to_upload):

        
    #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:

        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # Import dataframe into MySQL
        import sqlalchemy
        database_username = config2['database_username']
        database_password = config2['database_password']
        database_ip       = config2['database_ip']
        database_name     = db_name
    
        database_connection = sqlalchemy.create_engine('mysql+pymysql://{0}:{1}@{2}/{3}'.
                                                       format(database_username, database_password, 
                                                              database_ip, database_name))
        table_to_upload.to_sql(con=database_connection, name= db_table_name, if_exists='replace')
        print "Table has been dropped successfully!"
            
        cur.close()
        connection.close()


In [56]:
UploadPredictionsToParenthood('ml_provider_type','test_table_2', test_df_2)

Table has been dropped successfully!


In [30]:
def DropTableInParenthood(db_name, table_to_drop):
        #table1 = str(db_name) + '.' + str(table_name)
    with sshtunnel.SSHTunnelForwarder(**ssh_config) as tunnel:
        
        connection = mysql.connector.connect(**mysql_config)
        cur = connection.cursor()
        
        # SQL Statement to delete a database
        sql = ('''DROP TABLE IF EXISTS {db}.{t};'''.format(db = db_name, t = table_to_drop))

        # Execute the create database SQL statment through the cursor instance
        cur.execute(sql)
        print "Table has been dropped successfully!"

        # SQL query string
        sqlQuery            = ('''SHOW TABLES IN {db};'''.format(db = db_name))

        # Execute the sqlQuery
        cur.execute(sqlQuery)
        
        print 'Tables in the database are:'
        for row in cur.fetchall():
            for i in range(0,len(row)):
                print row[i]
    
        connection.close()

In [33]:
DropTableInParenthood('ml_provider_type','test_table_2')

Table has been dropped successfully!
Tables in the database are:
final_table
first_list
step_80053
step_82728
step_83550
step_83615
step_96361
step_96367
step_96413
step_G0008
step_J3490
step_Q2037


Reference: https://stackoverflow.com/questions/16476413/how-to-insert-pandas-dataframe-via-mysqldb-into-database
https://www.youtube.com/watch?v=i35OSGXt7wk https://pythontic.com/database/mysql/drop%20database